# 2MARKOWITZ: OPTIMIZACIÓN CLÁSICA DE MARKOWITZ

Este notebook demuestra el funcionamiento del módulo 2markowitz, que implementa las técnicas clásicas de optimización de carteras basadas en la teoría de Markowitz.

In [ ]:
import sys
import importlib
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importar módulos (los nombres que empiezan con números requieren importlib)
datos = importlib.import_module('1datos')
markowitz = importlib.import_module('2markowitz')

print("Librerías importadas correctamente")

## Paso 1: Cargar y preparar datos

In [ ]:
ruta_csv = '../data/prod_long_sharpe_u50_20260116_v5_train_dataset.csv'
retornos = datos.cargar_retornos(ruta_csv)

preparador = datos.PreparadorDatos(retornos, rf_anual=0.02)
preparador.calcular_estadisticas()
mu, Sigma, rf = preparador.obtener_estadisticas()

print(f"Datos preparados: μ shape {mu.shape}, Σ shape {Sigma.shape}, rf = {rf:.4f}")

## Paso 2: Optimización Markowitz con diferentes λ

In [ ]:
lambdas = [0.5, 1.0, 2.0, 5.0]
resultados_lambda = []

for lam in lambdas:
    res = markowitz.optimizar_markowitz_lambda(mu, Sigma, rf, lam)
    if res:
        resultados_lambda.append({
            'lambda': lam,
            'sharpe': res['sharpe'],
            'rentabilidad': res['rentabilidad'],
            'volatilidad': res['volatilidad']
        })
        print(f"λ={lam}: Sharpe={res['sharpe']:.4f}, Ret={res['rentabilidad']*100:.2f}%, Vol={res['volatilidad']*100:.2f}%")

df_lambda = pd.DataFrame(resultados_lambda)
print("\nResumen:")
print(df_lambda)

## Paso 3: Optimización de Máximo Sharpe

In [ ]:
cartera_max_sharpe = markowitz.optimizar_sharpe_maximo(mu, Sigma, rf)

if cartera_max_sharpe:
    print("Cartera de Máximo Sharpe:")
    print(f"  Sharpe Ratio: {cartera_max_sharpe['sharpe']:.4f}")
    print(f"  Rentabilidad: {cartera_max_sharpe['rentabilidad']*100:.2f}%")
    print(f"  Volatilidad: {cartera_max_sharpe['volatilidad']*100:.2f}%")
    print(f"  Peso en RF: {cartera_max_sharpe['w_rf']*100:.2f}%")
    print(f"  Número de activos con peso >1%: {np.sum(cartera_max_sharpe['w'] > 0.01)}")

## Paso 4: Visualizar distribución de pesos

In [ ]:
if cartera_max_sharpe:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Top 15 activos por peso
    w_sorted = np.sort(cartera_max_sharpe['w'])[::-1]
    top15 = w_sorted[:15]
    
    axes[0].barh(range(len(top15)), top15 * 100)
    axes[0].set_yticks(range(len(top15)))
    axes[0].set_yticklabels([f'Activo {i+1}' for i in range(len(top15))])
    axes[0].set_xlabel('Peso (%)')
    axes[0].set_title('Top 15 Activos por Peso en Cartera')
    axes[0].grid(True, alpha=0.3)
    
    # Distribución de pesos
    axes[1].hist(cartera_max_sharpe['w'] * 100, bins=20, edgecolor='black')
    axes[1].set_xlabel('Peso (%)')
    axes[1].set_ylabel('Frecuencia')
    axes[1].set_title('Distribución de Pesos')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('outputs/modulo2_pesos.png', dpi=300, bbox_inches='tight')
    plt.show()

## Paso 5: Construir Frontera Eficiente

In [ ]:
frontera_df = markowitz.construir_frontera_eficiente(mu, Sigma, rf, n_puntos=50)

print(f"Frontera eficiente construida con {len(frontera_df)} puntos")
print(f"  Sharpe máximo en frontera: {frontera_df['sharpe'].max():.4f}")
print(f"  Rentabilidad máxima: {frontera_df['rentabilidad'].max()*100:.2f}%")
print(f"  Volatilidad mínima: {frontera_df['volatilidad'].min()*100:.2f}%")

## Paso 6: Visualizar Frontera Eficiente

In [ ]:
fig = markowitz.visualizar_frontera_eficiente(frontera_df, cartera_max_sharpe, 
                                     ruta_guardado='outputs/modulo2_frontera.png')
plt.show()

## Paso 7: Análisis de Sensibilidad Temporal

In [ ]:
sensibilidad = markowitz.analizar_sensibilidad_temporal(retornos, rf, ventanas=[252, 504, 756, None])

print("Análisis de Sensibilidad Temporal:")
print(sensibilidad)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(sensibilidad['ventana'], sensibilidad['sharpe'])
axes[0].set_xlabel('Ventana Temporal')
axes[0].set_ylabel('Sharpe Ratio')
axes[0].set_title('Sharpe Ratio por Ventana Temporal')
axes[0].grid(True, alpha=0.3)

axes[1].bar(sensibilidad['ventana'], sensibilidad['concentracion'])
axes[1].set_xlabel('Ventana Temporal')
axes[1].set_ylabel('Índice de Concentración')
axes[1].set_title('Concentración por Ventana Temporal')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/modulo2_sensibilidad.png', dpi=300, bbox_inches='tight')
plt.show()

## Resumen del módulo 2markowitz

El módulo 2markowitz ha completado exitosamente:

1. Optimización de Markowitz con diferentes parámetros λ
2. Optimización directa del Sharpe Ratio máximo
3. Construcción de la frontera eficiente
4. Análisis de sensibilidad temporal

La cartera de máximo Sharpe está lista para ser comparada con estrategias multifactoriales en los siguientes módulos.